# Classifier

Dieses Notebook ist ein Beispiel, wie man die gespeicherten Gewichte des Modells, welche beim Training erstellt wurden, lädt und auf einem Datensatz testet.

## Imports

- Torch - Laden und nutzen des Modells \
- Matplotlib - Anzeigen der Bilder \
- OS - Dateipfadverwaltung \
- Random - Zufällige Auswahl der Bilder

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import os
import random

## Datensatz laden

Den Datensatz laden wir aus Google Drive, da es mit Colab am besten funktioniert.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/Dataset/data.zip /content
!unzip /content/data.zip

## Modell laden

Hier nochmal die gleiche Modell-Architektur wie im Training. Die Gewichte werden aus der pth-Datei, welche beim Training erstellt wurde geladen. Danach ist das Modell bereit zum Klassifizieren.

In [ ]:
IMAGE_SIZE = 256

In [ ]:
# Modell Klasse
class CNNModel(nn.Module):
    """
    TinyVGG:
    https://poloclub.github.io/cnn-explainer/
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2)
        )
        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(in_features=hidden_units * (IMAGE_SIZE//4) * (IMAGE_SIZE//4),
                    out_features=output_shape)
        )


    def forward(self, x: torch.Tensor):
        x = self.block_1(x)
        x = self.block_2(x)
        x = self.classifier(x)
        return x

In [ ]:
# Gewichte laden
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CNNModel(input_shape=3, hidden_units=32, output_shape=2).to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/Dataset/saves/model_weights_95.pth"))
model.eval()

## Ergebnis ausgeben

Im letzten Teil werden Bilder aus dem Datensatz zufällig ausgewählt und klassifiziert. Das Ergebnis wird dann grafisch dargestellt.

In [ ]:
# Verzeichnisse
base_dir = "/content/data"
classes = ["fake", "real"]
num_images_per_class = 5

# Transformation Training
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Bilder sammeln
selected_images = []
for label in classes:
    folder_path = os.path.join(base_dir, label)
    images = [os.path.join(folder_path, img) for img in os.listdir(folder_path) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
    selected = random.sample(images, num_images_per_class)
    selected_images.extend([(img, label) for img in selected])

In [ ]:
# Bilder mit Label und Vorhersage ausgeben
for img_path, true_label in selected_images:
    image = Image.open(img_path).convert("RGB")
    input_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        predicted_class = torch.argmax(output, dim=1).item()
        predicted_label = classes[predicted_class]

    # Anzeige mit Matplotlib
    plt.figure(figsize=(4, 4))
    plt.imshow(image)
    plt.axis("off")
    plt.title(f"Label: {true_label}\nVorhersage: {predicted_label}", fontsize=15)
    plt.show()